## Emission Intensity & Final Database

<p>The ECB portfolio is mapped with the CDP emissions data. However, out of 294 companies, 112 companies' emissions were not available in the data. To solve this, emission intensity was calculated based on the sector in proportion to the revenue of the companies. </p>

### Libraries to import

In [1]:
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")
from functions_file_3 import match_data,match_emissions,match_emissions_cdp,intensity,emissions_portfolio
pd.options.display.float_format = '{:.2f}'.format

In [2]:
directory = os.getcwd()
url = directory[:-7] + '2. Data/'

### Import datasets

In [3]:
# import ECB portfolio with all company information
portfolio = pd.read_excel(url + 'ECB Portfolio - Company Information.xlsx')

# emissions data
emissions_data = pd.read_excel(url + '3. Emissions Data - CDP/Emissions_Data.xlsx')

In [4]:
# import mappings to calculate emission intensity
dictionary_for_cdp = pd.read_excel(url + '3. Emissions Data - CDP/Emissions_Mapping.xlsx',sheet_name = 'Dictionary_for_CDP',skiprows=4)
cdp_companies = pd.read_excel(url + '3. Emissions Data - CDP/Emissions_Mapping.xlsx',sheet_name = 'CDP_Companies',skiprows = 4,usecols="A:G")


### Companies matched with CDP emissions data

In [5]:
matched_data, unmatched_data = match_data(portfolio,dictionary_for_cdp)

In [6]:
matched_emissions = match_emissions(matched_data,emissions_data)

In [7]:
matched_emissions.head()

,ISSUER_NAME,ACCOUNT_ID,GLOBAL_SCOPE1,GLOBAL_SCOPE2: LOCATION-BASED,GLOBAL_SCOPE2: MARKET-BASED,TOTAL_GHG
0,Delhaize Group S.A.,21338,1940000.00,2420000.00,0.00,4360000.00
1,Cofinimmo S.A./N.V.,31355,87.00,226.00,0.00,313.00
2,BEFIMMO SA,31360,265.02,42.15,0.00,307.17
3,Solvay S.A.,17296,10883951.00,2320421.00,2492389.00,13204372.00
4,Anheuser-Busch InBev S.A./N.V.,22350,2976744.00,1704692.00,0.00,4681436.00


### Emission Intensity based on NACE sector code

In [8]:
# merge emissions data with companies in CDP data
cdp_emissions = match_emissions_cdp(cdp_companies,emissions_data)

In [9]:
# calculate emissions intensity based on NACE code of sectors
emission_intensity = intensity(cdp_emissions)

### Companies unmatched with CDP emissions data

In [10]:
# unmatched data with emission intensity
data = unmatched_data.merge(emission_intensity, on = "NACE_CODE", how = "left")

### ECB Portfolio with emissions data

In [11]:
final_data = emissions_portfolio(portfolio, matched_emissions,data)
final_data.head()

,NCB,ISIN_CODE,ISSUER_NAME,MATURITY_DATE,COUPON_RATE,PUBLISHED_DATE,YEAR,MONTH,TOTAL__REVENUE,SIC_CODE,NACE_CODE,ECONOMIC_SECTOR,QUARTER,COUNTRY_OF_RISK,TOTAL_GHG: MEAN_INTENSITY,TOTAL_GHG: MEDIAN_INTENSITY
0,BE,BE0002178441,Delhaize Group S.A.,19/10/2018,4.25,2018-06-08,2018,6,71057.21,5411,G,Beverages,2018Q2,Belgium,4360000.00,4360000.00
1,BE,BE0002189554,Delhaize Group S.A.,27/02/2020,3.125,2018-06-08,2018,6,71057.21,5411,G,Beverages,2018Q2,Belgium,4360000.00,4360000.00
2,BE,BE0002256254,RESA SA,22/07/2026,1,2018-06-08,2018,6,393.06,4931,D,Utilities,2018Q2,Belgium,1143478.36,430738.73
3,BE,BE0002269380,Cofinimmo S.A./N.V.,09/12/2024,2,2018-06-08,2018,6,297.38,6531,L,Real estate,2018Q2,Belgium,313.00,313.00
4,BE,BE0002276450,Elia Transmission Belgium,07/04/2027,1.375,2018-06-08,2018,6,913.16,4911,D,Utilities,2018Q2,Belgium,2656537.68,1000695.52


### Download the file

In [12]:
final_data.to_excel(url + 'Master_File.xlsx',index = None)